In [5]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 18.7MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!pip install xgboost

    100% |████████████████████████████████| 114.0MB 472kB/s eta 0:00:01    94% |██████████████████████████████▏ | 107.5MB 53.3MB/s eta 0:00:01
You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

X_train = pd.read_csv('data/train_features.csv')
X_test = pd.read_csv('data/test_features.csv')
y_train = pd.read_csv('data/train_labels.csv')['charged_off']
sample_submission = pd.read_csv('data/sample_submission.csv')

X_train.shape, X_test.shape, y_train.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=40000, 
    random_state=42, stratify=y_train)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

def wrangle(X):
    X = X.copy()
    
    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns=['member_id', 'url', 'desc'])  # All null
    X = X.drop(columns='title')  # Duplicative of purpose
    X = X.drop(columns='grade')  # Duplicative of sub_grade
    
    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)

    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
        
    # Transform earliest_cr_line to an integer: how many days it's been open
    X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
    X['earliest_cr_line'] = pd.Timestamp.today() - X['earliest_cr_line']
    X['earliest_cr_line'] = X['earliest_cr_line'].dt.days
    
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)
    
    # Drop categoricals with high cardinality
    X = X.drop(columns=['emp_title', 'zip_code'])
    
    # Transform features with many nulls to binary flags
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

    for col in many_nulls:
        X[col] = X[col].isnull()
    
    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mean())
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(X_train)
X_test  = wrangle(X_test)
X_val  = wrangle(X_val)

X_train.shape, X_test.shape, X_val.shape

In [2]:
def submission(pipeline, X_train, y_train, X_test, name):
    pipe.fit(X_train, y_train)
    print(roc_auc_score(y_val, pipe.predict_proba(X_val)[:, 1]))
    sample_submission = pd.read_csv('data/sample_submission.csv') 
    submission = sample_submission.copy() 
    submission['charged_off'] = pipeline.predict_proba(X_test)[:, 1] 
    submission.to_csv( name, index=False)

In [3]:
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [4]:
################################ 
submission_url = '5_XgboostClassifier5a.csv'

pipe = make_pipeline(
    ce.OrdinalEncoder(), 
    XGBClassifier(
        learning_rate = 0.1, 
        n_estimators=200,
        eval_metric = "auc",
        xgbclassifier__silent = False,
        n_jobs=-1)
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', verbose=10)
submission(pipe, X_train, y_train, X_test, submission_url)


[CV]  ................................................................
[CV] ....................... , score=0.7330665561586248, total= 4.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


[CV] ....................... , score=0.7276338386169755, total= 4.5min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.9min remaining:    0.0s


[CV] ....................... , score=0.7282776203628762, total= 4.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 13.4min remaining:    0.0s


[CV] ....................... , score=0.7306482479064141, total= 4.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 17.7min remaining:    0.0s


[CV] ........................ , score=0.731095550701985, total= 4.4min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 22.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 22.1min finished


In [7]:
################################# 
# Add RobustScaler
submission_url = '5_XgboostClassifier5b.csv'

from sklearn.preprocessing import RobustScaler
pipe = make_pipeline(
    ce.OrdinalEncoder(),
    RobustScaler(),
    XGBClassifier(
        learning_rate = 0.1, 
        n_estimators=200,
        eval_metric = "auc",
        xgbclassifier__silent = False,
        n_jobs=-1)

)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', verbose=10)
submission(pipe, X_train, y_train, X_test, submission_url)


[CV]  ................................................................
[CV] ....................... , score=0.7321601325501664, total= 3.6min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.6min remaining:    0.0s


[CV] ....................... , score=0.7290753804139906, total= 3.6min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.1min remaining:    0.0s


[CV] ....................... , score=0.7297571157431308, total= 3.6min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 10.7min remaining:    0.0s


[CV] ........................ , score=0.731738175292772, total= 3.6min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 14.3min remaining:    0.0s


[CV] ....................... , score=0.7320241895878264, total= 3.6min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 17.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 17.8min finished
